In [56]:
import numpy as np
import folium
import requests

# Initial parameters
center_location = [33.755489, -84.401993]  # Latitude and Longitude for Atlanta, GA, near Mercedes-Benz Stadium
overall_radius = 1000  # The overall search radius in meters
tile_radius = 100  # The radius for each individual query in meters
cost_per_1000_requests = 17.00  # Update with the SDK cost you are using in USD

def meters_to_decimal_degrees(meters, latitude):
    """Convert meters to decimal degrees based on latitude."""
    return meters / (111.32 * 1000 * np.cos(latitude * (np.pi / 180)))

def create_location_grid_with_overlap(center_location, overall_radius, tile_radius, overlap=0.67):
    """Create a grid of locations with overlapping circular tiles for thorough coverage."""
    lat, lng = center_location
    delta_deg = meters_to_decimal_degrees(overall_radius, lat)
    adjusted_tile_radius = tile_radius * 2 * overlap
    tile_delta_deg = meters_to_decimal_degrees(adjusted_tile_radius, lat)
    
    lat_steps = int(np.ceil(delta_deg / tile_delta_deg))
    lng_steps = int(np.ceil(delta_deg / tile_delta_deg))
    
    locations = []
    for i in range(-lat_steps, lat_steps + 1):
        for j in range(-lng_steps, lng_steps + 1):
            new_lat = lat + (i * tile_delta_deg)
            new_lng = lng + (j * tile_delta_deg)
            locations.append((new_lat, new_lng))
    return locations

def query_google_places(latitude, longitude, radius, api_key):
    """Query Google Places API and return unique place IDs in the vicinity."""
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={latitude},{longitude}&radius={radius}&key={api_key}"
    response = requests.get(url)
    data = response.json()
    place_ids = {result['place_id'] for result in data.get('results', [])}
    return place_ids

# API Key for Google Places - Replace with your actual API key
api_key = 'yourapikey'

grid_coordinates_with_overlap = create_location_grid_with_overlap(center_location, overall_radius, tile_radius)

# Create a Folium map centered at the center_location
m = folium.Map(location=center_location, zoom_start=15)

# Add a circle for each coordinate in the grid to the map
for location in grid_coordinates_with_overlap:
    folium.Circle(
        location=(location[0], location[1]),
        radius=tile_radius,
        color='purple',
        fill=True,
        fill_opacity=0.4
    ).add_to(m)

# Save the map to an HTML file
m.save('C:/Users/agutierrez/Downloads/tiledgridsearchcoordinates.html')

# Initialize a set to keep track of unique place IDs
unique_place_ids = set()

# Iterate over each coordinate in the grid, query Google Places, and update the set of unique place IDs
for coordinate in grid_coordinates_with_overlap:
    latitude, longitude = coordinate
    place_ids = query_google_places(latitude, longitude, tile_radius, api_key)
    unique_place_ids.update(place_ids)

# Calculate the total number of unique places and the estimated cost
num_unique_places = len(unique_place_ids)
total_estimated_cost = (num_unique_places / 1000) * cost_per_1000_requests

print(f"Total number of unique places: {num_unique_places}")
print(f"Total estimated cost for these requests: ${total_estimated_cost:.2f}")


Total number of unique places: 2042
Total estimated cost for these requests: $34.71


In [55]:
import requests
import pandas as pd
import numpy as np
import time

# Your API Key and initial parameters
api_key = 'yourapikey'  # Make sure to replace this with your actual API key
center_location = '33.755489, -84.401993'  # Mercedes Benz Stadium, Atlanta, GA
overall_radius = 1000  # Larger area to cover in meters
tile_radius = 100  # Radius for each query in meters
type = 'restaurant'

def meters_to_decimal_degrees(meters, latitude):
    """Convert meters to decimal degrees based on latitude."""
    return meters / (111.32 * 1000 * np.cos(latitude * (np.pi / 180)))

def create_location_grid(center_location, overall_radius, tile_radius, overlap=0.67):
    """Create a grid of locations to cover the search area effectively with overlapping circles."""
    print("Creating location grid...")
    lat, lng = [float(coord) for coord in center_location.split(',')]
    delta_deg = meters_to_decimal_degrees(overall_radius, lat)
    
    adjusted_tile_radius = tile_radius * 2 * overlap  # Adjust tile_delta_deg for overlap
    tile_delta_deg = meters_to_decimal_degrees(adjusted_tile_radius, lat)
    
    lat_steps = int(np.ceil(delta_deg / tile_delta_deg))
    lng_steps = int(np.ceil(delta_deg / tile_delta_deg))
    
    locations = []
    for i in range(-lat_steps, lat_steps + 1):
        for j in range(-lng_steps, lng_steps + 1):
            new_lat = lat + (i * tile_delta_deg)
            new_lng = lng + (j * tile_delta_deg)
            locations.append(f"{new_lat},{new_lng}")
    print(f"Grid created with {len(locations)} locations.")
    return locations

def get_place_details(place_id, api_key):
    """Fetch place details such as the address using the Place Details API."""
    print(f"Fetching details for place ID: {place_id}")
    details_url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=formatted_address&key={api_key}"
    response = requests.get(details_url)
    details_data = response.json()
    if details_data.get('status') == 'OK':
        address = details_data['result'].get('formatted_address', '')
        return address
    return ""

def fetch_places_for_tile(api_key, location, radius, type, seen_place_ids):
    """Fetch places for a given tile and type, adding unique ones based on place_id."""
    print(f"Fetching places for location: {location}")
    places = []
    endpoint_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        'location': location,
        'radius': radius,
        'type': type,
        'key': api_key
    }

    while True:
        response = requests.get(endpoint_url, params=params)
        data = response.json()

        if data['status'] == 'OK':
            for place in data['results']:
                place_id = place.get('place_id')
                if place_id not in seen_place_ids:
                    seen_place_ids.add(place_id)
                    address = get_place_details(place_id, api_key)  # Fetch address only if unique
                    business_status = place.get('business_status', 'N/A')
                    types = ', '.join(place.get('types', []))  # Convert list of types to a string
                    place_info = {
                        'name': place.get('name'),
                        'latitude': place.get('geometry', {}).get('location', {}).get('lat'),
                        'longitude': place.get('geometry', {}).get('location', {}).get('lng'),
                        'address': address,
                        'place_id': place_id,
                        'business_status': business_status,
                        'types': types
                    }
                    places.append(place_info)

            next_page_token = data.get('next_page_token')
            if next_page_token:
                params['pagetoken'] = next_page_token
                params.pop('location', None)
                params.pop('radius', None)
                print("Waiting for next page token...")
                time.sleep(2)
            else:
                print("Finished fetching places for current location.")
                break
        else:
            print(f"Status not OK: {data['status']}")
            if 'error_message' in data:
                print(f"Error message: {data['error_message']}")
            break

    return places

def fetch_all_places(api_key, center_location, overall_radius, tile_radius, type):
    """Fetch all places within the overall radius by processing each tile."""
    print("Starting to fetch all places...")
    all_places = []
    seen_place_ids = set()
    locations = create_location_grid(center_location, overall_radius, tile_radius)
    
    for idx, location in enumerate(locations):
        print(f"Processing location {idx+1}/{len(locations)}...")
        tile_places = fetch_places_for_tile(api_key, location, tile_radius, type, seen_place_ids)
        all_places.extend(tile_places)
        time.sleep(0.1)  # Manage API call rate

    print(f"Finished fetching. Total unique places found: {len(all_places)}")
    return all_places

# Fetch all places with geographical tiling
all_places = fetch_all_places(api_key, center_location, overall_radius, tile_radius, type)

# Convert the list of places into a DataFrame
df = pd.DataFrame(all_places)

# Export the DataFrame to an Excel file
file_name = 'C:/Users/agutierrez/Downloads/AtlantaRestaurants1km.xlsx'  # Adjust the path as needed
df.to_excel(file_name, index=False)

print(f"Data saved to {file_name}")


Starting to fetch all places...
Creating location grid...
Grid created with 289 locations.
Processing location 1/289...
Fetching places for location: 33.74338786108733,-84.41409413891267
Status not OK: ZERO_RESULTS
Processing location 2/289...
Fetching places for location: 33.74338786108733,-84.4125814965486
Status not OK: ZERO_RESULTS
Processing location 3/289...
Fetching places for location: 33.74338786108733,-84.4110688541845
Status not OK: ZERO_RESULTS
Processing location 4/289...
Fetching places for location: 33.74338786108733,-84.40955621182043
Status not OK: ZERO_RESULTS
Processing location 5/289...
Fetching places for location: 33.74338786108733,-84.40804356945634
Fetching details for place ID: ChIJZ9C2CW4D9YgR8E0icqv2sN8
Finished fetching places for current location.
Processing location 6/289...
Fetching places for location: 33.74338786108733,-84.40653092709226
Status not OK: ZERO_RESULTS
Processing location 7/289...
Fetching places for location: 33.74338786108733,-84.40501828